In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import numpy as np
import pandas as pd
from transformers import BertTokenizer
import nltk
import string
import re
import sys
from bs4 import BeautifulSoup
import requests
import time
import spacy

In [2]:
options = Options()
options.add_argument('headless');
driver = webdriver.Chrome(executable_path=r'D:\Downloads\chromedriver_win32\chromedriver.exe', options = options)

In [3]:
skills = ['Deep Learning', 'Convolutional Neural Networks', 'TensorFlow', 'Machine Learning','Algorithms','Computer Vision','Deep Reinforcement Learning','Computer Science','Python','Matlab','Recurrent Neural Networks','Django','Java','Keras','PyTorch','Neural Networks','Edge Computing','OpenCV','Scikit-Learn','Natural Language Processing','Object Detection','Reinforcement Learning','Generative Adverserial Networks']
skills_df = pd.read_csv("../data/skills.csv")
skills_df = skills_df.drop(columns = ["Unnamed: 0"])
skills_df.head(25)

,skills,summary
0,Deep Learning,Deep learning (also known as deep structured ...
1,Convolutional Neural Networks,"In deep learning, a convolutional neural netwo..."
2,TensorFlow,TensorFlow is a free and open-source software ...
3,Machine Learning,Machine learning (ML) is the study of computer...
4,Algorithms,"In mathematics and computer science, an algori..."
5,Computer Vision,Computer vision is an interdisciplinary scient...
6,Deep Reinforcement Learning,Deep reinforcement learning (deep RL) is a sub...
7,Computer Science,"Computer science is the study of computation, ..."
8,Python,NaN
9,Matlab,"MATLAB (an abbreviation of ""MATrix LABoratory""..."


#### Using Bert Tokenizer

In [4]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
def bertTokens(text):
    bertTokens = bert_tokenizer.tokenize(text)
    return bertTokens

In [18]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

def remove_punctuation(text):
    try:
        punctuationfree = "".join([i for i in text if i not in string.punctuation])
    except:
        return " "
    return punctuationfree

def lower_case(text):
    try:
        lower = text.lower()
    except:
        print("Lower Case failed?")
        return ""
    return lower

def tokenization(text):
    tokens = nltk.word_tokenize(text)
    return tokens

In [19]:
lemmatization(["Learning"])

['learn']

In [20]:
bert_vocab = list(bert_tokenizer.vocab.keys())
def rare_word_extractor(word_list):
    rare_words = {}
    for word in word_list:
        if word in bert_vocab:
            ind = bert_vocab.index(word)
            if ind > 2500:
                rare_words[word] = ind
        else:
            rare_words[word] = 0
    rare_words_list = list(rare_words.keys())
    return rare_words_list
skills_df['clean_summary'] = skills_df['summary'].apply(lambda x:remove_punctuation(x)) 
skills_df['clean_summary'] = skills_df['clean_summary'].apply(lambda x:lower_case(x)) 
skills_df['tokens'] = skills_df['clean_summary'].apply(lambda x:tokenization(x)) 
skills_df['lemma_tokens'] = skills_df['tokens'].apply(lambda x:lemmatization(x)) 
skills_df['rare_words'] = skills_df['lemma_tokens'].apply(lambda x:rare_word_extractor(x)) 
skills_df.head()

,skills,summary,clean_summary,tokens,lemma_tokens,rare_words
0,Deep Learning,Deep learning (also known as deep structured ...,deep learning also known as deep structured l...,"[deep, learning, also, known, as, deep, struct...","[deep, learn, also, know, as, deep, structure,...","[deep, learn, structure, broad, machine, metho..."
1,Convolutional Neural Networks,"In deep learning, a convolutional neural netwo...",in deep learning a convolutional neural networ...,"[in, deep, learning, a, convolutional, neural,...","[in, deep, learn, a, convolutional, neural, ne...","[deep, learn, convolutional, neural, network, ..."
2,TensorFlow,TensorFlow is a free and open-source software ...,tensorflow is a free and opensource software l...,"[tensorflow, is, a, free, and, opensource, sof...","[tensorflow, be, a, free, and, opensource, sof...","[tensorflow, opensource, software, library, ma..."
3,Machine Learning,Machine learning (ML) is the study of computer...,machine learning ml is the study of computer a...,"[machine, learning, ml, is, the, study, of, co...","[machine, learn, ml, be, the, study, of, compu...","[machine, learn, ml, study, computer, algorith..."
4,Algorithms,"In mathematics and computer science, an algori...",in mathematics and computer science an algorit...,"[in, mathematics, and, computer, science, an, ...","[in, mathematic, and, computer, science, an, a...","[mathematic, computer, science, algorithm, lis..."


In [22]:
history = pd.read_csv("../data/web_history/with_labels.csv")
history = history.drop(columns = ["Unnamed: 0"])
history

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 35: invalid start byte

In [ ]:
history.LABEL.describe()

In [ ]:
history.LABEL.unique()

In [ ]:
history.head()

In [ ]:
history['H1'] = pd.Series()
history['H2'] = pd.Series()
history['H3'] = pd.Series()
history['p'] = pd.Series()

In [ ]:
def process_url(url):
    driver.get(url)
    h1 = driver.find_elements_by_tag_name("h1")
    h2 = driver.find_elements_by_tag_name("h2")
    h3 = driver.find_elements_by_tag_name("h3")
    p = driver.find_elements_by_tag_name("p")
    return (h1,h2,h3,p)

def extract_rare_words_from_sel_objects(tag):
    rare_words = []
    if tag != None:
        for text in tag:
            rare_words.append(rare_word_extractor(tokenization(lower_case(remove_punctuation(text.text)))))
    return rare_words

In [ ]:
for index,row in history.iterrows():
    print("URL INDEX: ", index, "  LABEL: ", row['LABEL'] )
    if index >= 1000:
        break
    
    if row['LABEL'] != "MISC":
        try:
            (h1,h2,h3,p) = process_url(row['URL'])
        except:
            print("URL: ", row['URL'], "\n\n")
            print("Index:", index,"Exception while processing url")
            row['H1'] = []
            row['H2'] = []
            row['H3'] = []
            row['p'] = []
            continue
        time.sleep(1)

        try:
            rare_words_h1 = extract_rare_words_from_sel_objects(h1)
        except:
            print("Index:", index,"Exception while extracting h1")
            rare_words_h1 = []

        try:
            rare_words_h2 = extract_rare_words_from_sel_objects(h2)
        except:
            print("Index:", index,"Exception while extracting h2")
            rare_words_h2 = []

        try:
            rare_words_h3 = extract_rare_words_from_sel_objects(h3)
        except:
            print("Index:", index,"Exception while extracting h2")
            rare_words_h3 = []

        try:
            rare_words_p  = extract_rare_words_from_sel_objects(p)
        except:      
            print("Index:", index,"Exception while extracting h2")
            rare_words_p = []
        
        print("URL: ", row['URL'], "\n\n")
        print("H1:  ",rare_words_h1, str("\n"))
        print("H2:  ",rare_words_h2,str("\n"))
        print("H3:  ",rare_words_h3,str("\n"))
        print("P:  ",rare_words_p,str("\n"))
    else:
        rare_words_h1 = []
        rare_words_h2 = []
        rare_words_h3 = []
        rare_words_p = []    
    
    row['H1'] = rare_words_h1
    row['H2'] = rare_words_h2
    row['H3'] = rare_words_h3
    row['p'] = rare_words_p

In [ ]:
history.head()

In [ ]:
history.H1.unique()

In [ ]:
len(history[history['LABEL'] == "WORK"])

In [ ]:
len(history[history['LABEL'] == "MISC"])

In [ ]:
len(history[history['LABEL'] == "SHOPPING"])

In [ ]:
len(history[history['LABEL'] == "TRAVEL"])

In [ ]:
len(history[history['LABEL'] == "ENTERTAINMENT"])

In [ ]:
h1,h2,h3,p = process_url("https://www.youtube.com/watch?v=4I3gS1cmqe4")
h1,h2,h3,p = process_url("https://www.nytimes.com/")

In [ ]:
extract_rare_words_from_sel_objects(h1)

In [ ]:
extract_rare_words_from_sel_objects(h2)

In [ ]:
extract_rare_words_from_sel_objects(h3)

In [ ]:
extract_rare_words_from_sel_objects(p)

In [ ]:
data = requests.get("https://www.youtube.com/watch?v=4I3gS1cmqe4")

In [ ]:
html = BeautifulSoup(data.text, 'html.parser')

In [ ]:
html.findAll(text = True)